In [1]:
import pandas as pd
import numpy as np
import os, time,json
import tmdbsimple as tmdb
from tqdm.notebook import tqdm_notebook
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['tmdb_api_results_2000.json',
 'final_tmdb_data_2000.csv.gz',
 'tmdb_api_results_2001.json',
 'title_basics.csv.gz',
 'final_tmdb_data_2001.csv.gz',
 '.ipynb_checkpoints',
 'title_akas.csv.gz',
 'titleR.csv.gz']

In [2]:
basics_url='https://datasets.imdbws.com/title.basics.tsv.gz'
title_url='https://datasets.imdbws.com/title.akas.tsv.gz'
title_ratings='https://datasets.imdbws.com/title.ratings.tsv.gz'

In [5]:
basics= pd.read_csv(basics_url, sep='\t', low_memory=False)



In [4]:
titlesakas= pd.read_csv(title_url, sep='\t', low_memory=False)

KeyboardInterrupt: 

In [ ]:
titleR= pd.read_csv(title_ratings, sep='\t', low_memory=False)

In [ ]:
basics.replace({'\\N':np.nan},inplace=True)
basics.isna().sum()

In [ ]:
basics.head()

In [ ]:
basics= basics.dropna(subset=['runtimeMinutes','startYear', 'genres'])

In [ ]:
basics.head()

In [ ]:
basics.startYear= basics.startYear.astype(float)

In [ ]:
basics = basics.loc[ basics['titleType']=='movie']

In [ ]:
basics.head()

In [ ]:
basics.info()


In [ ]:
basics = basics[(basics['startYear']>=2000)&(basics['startYear']<2022)]
basics


In [ ]:
# Exclude movies that are included in the documentary category.
is_documentary = basics['genres'].str.contains('documentary',case=False)
basics = basics[~is_documentary]


## cleaning for akas

In [ ]:
titlesakas.replace({'\\N':np.nan},inplace=True)
titlesakas.isna().sum()

In [ ]:
titlesakas=titlesakas[titlesakas.region=="US"]

In [ ]:
# Filter the basics table down to only include the US by using the filter akas dataframe
keepers =basics['tconst'].isin(titlesakas['titleId'])
keepers


In [ ]:
basics = basics[keepers]
basics


In [ ]:
titleR.replace({'\\N':np.nan},inplace=True)
titleR.isna().sum()

In [ ]:
keepers = titleR['tconst'].isin(titlesakas['titleId'])
ratings = titleR[keepers]
ratings

In [ ]:
# example making new folder with os
import os
os.makedirs('Data/',exist_ok=True) 
# Confirm folder created
os.listdir("Data/")

In [ ]:
## Save current dataframe to file.
basics.to_csv("Data/title_basics.csv.gz",compression='gzip',index=False)


In [ ]:
## Save current dataframe to file.
titlesakas.to_csv("Data/title_akas.csv.gz",compression='gzip',index=False)


In [ ]:
## Save current dataframe to file.
ratings.to_csv("Data/titleR.csv.gz",compression='gzip',index=False)


In [ ]:
!pip install tmdbsimple

In [ ]:
with open('/Users/alisonwilliams/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
login.keys()

In [ ]:
tmdb.API_KEY =  login['api-key']

In [ ]:
movie = tmdb.Movies()

In [ ]:
info = movie.info

In [ ]:
YEARS_TO_GET = [2000,2001]

In [ ]:
def get_movie_with_rating(movie_id):
    """Adapted from source = https://github.com/celiao/tmdbsimple"""
    #Get the movie object for the current id
    movie = tmdb.Movies(movie_id)
    
    #save the info releases dictionaries
    info = movie.info()
    releases = movie.releases()
    
    #Loop through countries in releases
    for c in releases ['countries']:
        #if the country abbreviation==Us
        if c['iso_3166_1'] =='US':
##save a "certification" key in info with the certification
            info ['certification'] = c['certification']
    return info

In [ ]:
errors = [ ]

In [ ]:
def write_json(new_data, filename):
    """Appends a list of records (new_data) to a json file(filename),
    Adapted from://www.geeksforgeeks.org/append-to-json-file-using-python/"""
    with open(filename,'r+') as file:
        #first we load existing file into a dict
        file_data = json.load(file)
        ##choose extend or append
        if (type(new_data) == list & type(file_data) == list):
            file_data.extend(new_data)
        else:
            file_data.append(new_data)
        # sets files current position at offset.
        file.seek(0)
        #convert back to json
        json.dump(file_data, file)

In [ ]:
# Start of OUTER loop
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
    #Defining the JSON file to store results for year
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
# Check if file exists
    file_exists = os.path.isfile(JSON_FILE)
# If it does not exist: create it
    if file_exists == False:
# save an empty dict with just "imdb_id" to the new json file.
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)
#Saving new year as the current df
    df = basics.loc[ basics['startYear']==YEAR].copy()
# saving movie ids to list
    movie_ids = df['tconst'].copy()
# Load existing data from json into a dataframe called "previous_df"
    previous_df = pd.read_json(JSON_FILE)
# filter out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]
    #Get index and movie id from list
    # INNER Loop
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
        try:
            # Retrieve then data for the movie id
            temp = get_movie_with_rating(movie_id)  
            # Append/extend results to existing file using a pre-made function
            write_json(temp,JSON_FILE)
            # Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)
            
        except Exception as e:
            errors.append([movie_id, e])
        final_year_df= pd.read_json(JSON_FILE)
        final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz",
                            compression="gzip", index=False)

In [ ]:
get_movie_with_rating('tt0848228')

In [ ]:
get_movie_with_rating('tt0332280')

In [ ]:
## create a col with a list of genres
df['genres_split'] = df['genres'].str.split(',')
df


In [ ]:
exploded_genres = df.explode('genres_split')
exploded_genres


In [ ]:
unique_genres = sorted(exploded_genres['genres_split'].unique())


In [ ]:
## save the tconst and genres_split as new df
title_genres = exploded_genres[['tconst', 'genres_split']].copy()
title_genres.head()

In [ ]:
## making the genre mapper dictionary
genre_ints = range(len(unique_genres))
genre_map = dict(zip(unique_genres, genre_ints))
genre_map

In [ ]:
## make new integer genre_id and drop string genres
df['genre_id'] = df['genres_split'].replace(genre_map)
df = df.drop(columns='genres_split')


In [ ]:
df.head()

In [ ]:
df = df.drop(columns = ['originalTitle', 'isAdult', 'endYear'])

In [ ]:
df.head()